## The Battle of Neighborhoods
  #### ——Potential locations for the sharing economy of power banks in New York


#### 1. Importing the necessary packages

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

#### 2. Importing the data

As in the lab, we will import the data directly from the server.

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### 3. Exploring the data

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Defining the neighbourhood data as nb_data

In [5]:
nb_data = newyork_data['features']

Taking a look at the first item in this list.

In [6]:
nb_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

This data is a dictionary (in fact a nested dictionary)

#### 4. Tranform the data into a *pandas* dataframe

Transforming this data in to a workable *pandas* dataframe.

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
nb = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [8]:
print(nb)
print('Yes, they look fine')

Empty DataFrame
Columns: [Borough, Neighborhood, Latitude, Longitude]
Index: []
Yes, they look fine


Filling in the dataframe one row each time with a for loop

In [9]:
for data in nb_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nb = nb.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Checking out the first 10 rows

In [10]:
nb.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nb['Borough'].unique()),
        nb.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### 5. Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### 6. Create a map of New York with neighborhoods superimposed on top.

In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nb['Latitude'], nb['Longitude'], nb['Borough'], nb['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

##### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### 7. Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'F4IHTT2S4ETNHW22R33KEKYWRISGMEDBGOJPODZAE4QPMV1W'
CLIENT_SECRET = 'XHHWUYHMKEYIGSK1BNZ140BX5XVHT4ON41LIZWRR3DFKIAVC'
VERSION = '20180605' # Foursquare API version

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: F4IHTT2S4ETNHW22R33KEKYWRISGMEDBGOJPODZAE4QPMV1W
CLIENT_SECRET:XHHWUYHMKEYIGSK1BNZ140BX5XVHT4ON41LIZWRR3DFKIAVC


#### 8. Let's explore the neighborhoods

In [15]:
nb['Neighborhood'].head()

0      Wakefield
1     Co-op City
2    Eastchester
3      Fieldston
4      Riverdale
Name: Neighborhood, dtype: object

In [16]:
print('New York has {} neighborhoods.'.format(
        nb.shape[0]
    )
)

New York has 306 neighborhoods.


Let's start with our first neighborhood —— Wakefield

In [17]:
neighborhood_latitude = nb.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = nb.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = nb.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


##### Let's explore the top 5 most popular places within 500m radius!

In [18]:
LIMIT = 10 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude,
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=F4IHTT2S4ETNHW22R33KEKYWRISGMEDBGOJPODZAE4QPMV1W&client_secret=XHHWUYHMKEYIGSK1BNZ140BX5XVHT4ON41LIZWRR3DFKIAVC&v=20180605&ll=40.89470517661,-73.84720052054902&radius=500&limit=10'

Send the GET request and examine the resutls

In [19]:
results = requests.get(url).json()

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Rite Aid,Pharmacy,40.896649,-73.844846
2,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568
3,Cooler Runnings Jamaican Restaurant Inc,Caribbean Restaurant,40.898276,-73.850381
4,Dunkin',Donut Shop,40.890459,-73.849089


#### 9. Explore all neighborhoods in New York

##### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Now create a new dataframe called *ny_venues*.

In [23]:
ny_venues = getNearbyVenues(names=nb['Neighborhood'],latitudes=nb['Latitude'],longitudes=nb['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

KeyError: 'groups'

##### Let's check the size of the resulting dataframe

In [ ]:
print('New York has {} Neighborhood and total {} venues.'.format(len(ny_venues['Neighborhood'].unique()),ny_venues.shape[0]))

In [ ]:
print(ny_venues.shape)
ny_venues.head()

#### 10. Analyze Each Neighborhood

In [ ]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

And let's examine the new dataframe size.

In [ ]:
ny_onehot.shape

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

#### Let's confirm the new size

In [24]:
ny_grouped.shape

NameError: name 'ny_grouped' is not defined

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for nb in ny_grouped['Neighborhood']:
    print("----"+nb+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == nb].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                  venue  freq
0           Pizza Place   0.2
1        Breakfast Spot   0.1
2            Donut Shop   0.1
3  Fast Food Restaurant   0.1
4    Chinese Restaurant   0.1


----Annadale----
         venue  freq
0  Pizza Place  0.22
1         Park  0.11
2   Sports Bar  0.11
3   Restaurant  0.11
4        Diner  0.11


----Arden Heights----
         venue  freq
0     Pharmacy  0.25
1  Coffee Shop  0.25
2  Pizza Place  0.25
3     Bus Stop  0.25
4  Yoga Studio  0.00


----Arlington----
                 venue  freq
0        Grocery Store  0.17
1        Deli / Bodega  0.17
2  American Restaurant  0.17
3             Bus Stop  0.17
4        Boat or Ferry  0.17


----Arrochar----
                   venue  freq
0     Italian Restaurant   0.2
1             Bagel Shop   0.1
2         Sandwich Place   0.1
3            Pizza Place   0.1
4  Outdoors & Recreation   0.1


----Arverne----
             venue  freq
0        Surf Spot   0.3
1   Sandwich Place   0.1
2       Donut Sh

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [42]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allerton,Pizza Place,Chinese Restaurant,Pharmacy,Breakfast Spot,Dessert Shop
1,Annadale,Pizza Place,Park,Food,Train Station,Pharmacy
2,Arden Heights,Pizza Place,Bus Stop,Pharmacy,Coffee Shop,Event Service
3,Arlington,Deli / Bodega,Food,American Restaurant,Grocery Store,Boat or Ferry
4,Arrochar,Italian Restaurant,Sandwich Place,Deli / Bodega,Mediterranean Restaurant,Bagel Shop


<a id='item4'></a>

#### 11. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [1]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

NameError: name 'ny_grouped' is not defined

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = nb

# merge nb_grouped with nb_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head()

ValueError: cannot insert Cluster Labels, already exists

Again, visualizing the clusters

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'ny_merged' is not defined

<a id='item5'></a>

#### Cluster 1

In [49]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

AttributeError: 'str' object has no attribute 'loc'

#### Cluster 2

In [ ]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

#### Cluster 3

In [ ]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

#### Cluster 4

In [ ]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

#### Cluster 5

In [ ]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]